Tabris Loveless
11/14/2022

In [23]:
import numpy as np
import requests
import scipy
from numba import jit

def get_pos_from_url(address='http://doye.chem.ox.ac.uk/jon/structures/LJ/points/', N=7):
    url_address = address + str(N)
    data_str = requests.get(url_address).text
    return parse_url_text(data_str)    
    
def parse_url_text(data_str):
    x_array = []
    text = data_str.split('\n')
    for line in text:
        [x_array.append(float(i)) for i in line.split()]
    return np.array(x_array)

@jit
def lj(r):   
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

@jit
def total_energy(positions):
    """
    Calculates the total energy of a given system
    """
    E = 0
    n_atom = int(len(positions)/3)
    
    #assuming [x0 y0, z0, x1, y1, z1]
    for i in range(n_atom-1):
        for j in range(i+1, n_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            dist = np.linalg.norm(pos1-pos2)
            E += lj(dist)
    return E

@jit
def init_pos(N, L = 5):
    return L*np.random.random_sample((N*3,))

def init_bounds(N, l, r):
    """
    Returns list of  bounds classes for given l and r and N number of atoms
    """
    result = []
    for i in range(N*3):
        bound = [l, r]
        result.append(bound)
    return result


In [60]:
#bounds set from 0 to 5
def lj_evo(l, r, N):
    atom_list = init_bounds(N, l, r)
    results = scipy.optimize.differential_evolution(total_energy, atom_list, strategy = 'best1exp', polish = True)

    url_results = get_pos_from_url(N = N)

    print('URL minumum:           ',total_energy(url_results))
    
    print('Scipy Differntial Evo: ', total_energy(results.x))
    print(results.x)

In [61]:
lj_evo(0, 10, 5)

URL minumum:            -9.103852415681363
Scipy Differntial Evo:  -9.103852415410929
[7.12953734 6.10754324 3.81846565 6.4232842  6.98036473 3.76387124
 6.66866379 6.39695592 2.83488805 6.03746808 5.93641917 3.63747826
 7.44352016 7.05349028 3.30733892]


In [62]:
lj_evo(0, 10, 13)

URL minumum:            -44.32680141873466
Scipy Differntial Evo:  -44.326801390568455
[5.52950132 7.65024777 5.64825203 4.56730594 7.74823326 4.08231959
 5.05089195 7.1316951  4.82825003 5.53804668 7.1967401  3.8644837
 4.56372686 7.0666562  5.79199911 5.53446451 6.51515554 5.57417029
 4.57006914 6.19190842 5.06486822 3.96906908 7.12721457 4.82419467
 6.13271421 7.13618516 4.83228896 5.53976498 6.23488126 4.47174339
 4.56202556 8.02851601 5.1847517  4.57229382 6.61315202 4.00822673
 5.53170309 8.07148485 4.5916132 ]
